In [71]:
import pandas as pd
from docx import Document
from collections import defaultdict
import re
import log_cardapio as log_setup

logger = log_setup.setup_logger()


In [ ]:
def processar_cardapio_completo(caminho_arquivo, indice_tabela=0):
    """
    Processa o cardápio do arquivo DOCX e retorna dados estruturados
    """
    try:
        logger.info(f"Iniciando processamento do arquivo: {caminho_arquivo}")
        logger.info(f"Índice da tabela: {indice_tabela}")
        
        doc = Document(caminho_arquivo)
        tabela = doc.tables[indice_tabela]
        
        logger.info(f"Tabela encontrada com {len(tabela.rows)} linhas")
        
        dados = []
        for i, linha in enumerate(tabela.rows):
            celulas = [celula.text.strip() for celula in linha.cells]
            dados.append(celulas)
            if i < 5:  # Log das primeiras 5 linhas para debug
                logger.debug(f"Linha {i}: {celulas}")
        
        # Padrão para identificar datas
        padrao_data = re.compile(r'.*(segunda|terça|quarta|quinta|sexta|sábado|domingo).*setembro.*2025', re.IGNORECASE)
        
        # Encontra todas as linhas com datas
        linhas_datas = []
        for i, linha in enumerate(dados):
            for celula in linha:
                if padrao_data.match(celula):
                    linhas_datas.append(i)
                    logger.info(f"Data encontrada na linha {i}: {celula}")
                    break
        
        if not linhas_datas:
            logger.warning("Nenhuma data encontrada no documento!")
            return []
        
        logger.info(f"Encontradas {len(linhas_datas)} datas no documento")
        
        # Processa cada dia
        cardapios = []
        categorias_validas = ['Saladas', 'Acompanhamento', 'Guarnição', 'Vegetariano', 'Prato Principal']
        
        for i, linha_data_idx in enumerate(linhas_datas):
            try:
                # Pega o bloco deste dia
                inicio = linha_data_idx
                fim = linhas_datas[i + 1] if i + 1 < len(linhas_datas) else len(dados)
                bloco_dia = dados[inicio:fim]
                
                # Extrai a data
                data = ''
                for celula in bloco_dia[0]:
                    if padrao_data.match(celula):
                        data = celula
                        break
                
                if not data:
                    logger.warning(f"Não foi possível extrair data do bloco {i}")
                    continue
                
                logger.info(f"Processando dia: {data}")
                
                # Processa as categorias
                cardapio_dia = {'data': data, 'categorias': {}}
                categoria_atual = None
                
                for j, linha in enumerate(bloco_dia[1:], start=1):
                    if not any(linha):  # Linha vazia
                        continue
                        
                    # Verifica se é uma nova categoria
                    primeira_celula = linha[0] if linha else ''
                    
                    if primeira_celula in categorias_validas:
                        categoria_atual = primeira_celula
                        cardapio_dia['categorias'][categoria_atual] = []
                        logger.debug(f"Nova categoria '{categoria_atual}' na linha {inicio + j}")
                        
                        # Adiciona pratos das colunas seguintes
                        for k, celula in enumerate(linha[1:], start=1):
                            if celula and celula not in categorias_validas:
                                cardapio_dia['categorias'][categoria_atual].append(celula)
                                logger.debug(f"  Prato '{celula}' adicionado à categoria '{categoria_atual}'")
                    
                    elif categoria_atual:
                        # Adiciona pratos à categoria atual
                        for k, celula in enumerate(linha[1:], start=1):
                            if celula and celula not in categorias_validas:
                                cardapio_dia['categorias'][categoria_atual].append(celula)
                                logger.debug(f"  Prato adicional '{celula}' adicionado à categoria '{categoria_atual}'")
                
                # Log do cardápio processado
                logger.info(f"Cardápio processado para {data}:")
                for categoria, pratos in cardapio_dia['categorias'].items():
                    logger.info(f"  {categoria}: {len(pratos)} pratos")
                
                cardapios.append(cardapio_dia)
                
            except Exception as e:
                logger.error(f"Erro ao processar dia {i}: {str(e)}")
                continue
        
        logger.info(f"Processamento concluído. Total de dias processados: {len(cardapios)}")
        return cardapios
        
    except Exception as e:
        logger.error(f"Erro geral no processamento: {str(e)}")
        raise

def cardapios_para_dataframe(cardapios):
    """
    Converte a lista de cardápios para DataFrame
    """
    try:
        logger.info("Convertendo cardápios para DataFrame")
        
        linhas = []
        
        for cardapio in cardapios:
            data = cardapio['data']
            
            for categoria, pratos in cardapio['categorias'].items():
                for ordem, prato in enumerate(pratos, start=1):
                    linhas.append({
                        'data': data,
                        'categoria': categoria,
                        'prato': prato,
                        'ordem': ordem
                    })
        
        df = pd.DataFrame(linhas)
        logger.info(f"DataFrame criado com {len(df)} linhas e {len(df.columns)} colunas")
        logger.info(f"Colunas: {list(df.columns)}")
        
        if not df.empty:
            logger.info("Primeiras 5 linhas do DataFrame:")
            for i, row in df.head().iterrows():
                logger.info(f"  Linha {i}: {row.to_dict()}")
        
        return df
        
    except Exception as e:
        logger.error(f"Erro ao converter para DataFrame: {str(e)}")
        raise

# Função principal
def processar_cardapio(caminho_arquivo):
    """
    Função principal para processar o cardápio
    """
    try:
        logger.info("=" * 60)
        logger.info("INICIANDO PROCESSAMENTO DE CARDÁPIO")
        logger.info("=" * 60)
        
        # Processa o cardápio
        cardapios = processar_cardapio_completo(caminho_arquivo)
        
        if not cardapios:
            logger.warning("Nenhum cardápio foi processado!")
            return pd.DataFrame()
        
        # Converte para DataFrame
        df = cardapios_para_dataframe(cardapios)
        
        logger.info("=" * 60)
        logger.info("PROCESSAMENTO CONCLUÍDO COM SUCESSO")
        logger.info("=" * 60)
        
        return df
        
    except Exception as e:
        logger.error(f"Erro na função principal: {str(e)}")
        return pd.DataFrame()



2025-09-03 14:33:51 (BRT) - INFO - ============================================================
2025-09-03 14:33:51 (BRT) - INFO - ============================================================
2025-09-03 14:33:51 (BRT) - INFO - ============================================================
2025-09-03 14:33:51 (BRT) - INFO - ============================================================
2025-09-03 14:33:51 (BRT) - INFO - ============================================================
2025-09-03 14:33:51 (BRT) - INFO - ============================================================
2025-09-03 14:33:51 (BRT) - INFO - ============================================================
2025-09-03 14:33:51 (BRT) - INFO - ============================================================
2025-09-03 14:33:51 (BRT) - INFO - ============================================================
2025-09-03 14:33:51 (BRT) - INFO - ============================================================
2025-09-03 14:33:51 (BRT) - INFO - =====

Nenhum dado foi processado.


In [ ]:
# Exibe resultado
data = processar_cardapio("Cardápio de mes setembro.docx")
df = cardapios_para_dataframe(data)
df

2025-09-03 14:33:51 (BRT) - INFO - Iniciando processamento do arquivo: Cardápio de mes setembro.docx
2025-09-03 14:33:51 (BRT) - INFO - Iniciando processamento do arquivo: Cardápio de mes setembro.docx
2025-09-03 14:33:51 (BRT) - INFO - Iniciando processamento do arquivo: Cardápio de mes setembro.docx
2025-09-03 14:33:51 (BRT) - INFO - Iniciando processamento do arquivo: Cardápio de mes setembro.docx
2025-09-03 14:33:51 (BRT) - INFO - Iniciando processamento do arquivo: Cardápio de mes setembro.docx
2025-09-03 14:33:51 (BRT) - INFO - Iniciando processamento do arquivo: Cardápio de mes setembro.docx
2025-09-03 14:33:51 (BRT) - INFO - Iniciando processamento do arquivo: Cardápio de mes setembro.docx
2025-09-03 14:33:51 (BRT) - INFO - Iniciando processamento do arquivo: Cardápio de mes setembro.docx
2025-09-03 14:33:51 (BRT) - INFO - Iniciando processamento do arquivo: Cardápio de mes setembro.docx
2025-09-03 14:33:51 (BRT) - INFO - Iniciando processamento do arquivo: Cardápio de mes sete

,data,categoria,prato,ordem
0,"segunda-feira, 01 setembro de 2025",Saladas,"Mix grão (milho, ervilha, grão de bico)",1
1,"segunda-feira, 01 setembro de 2025",Saladas,"Mix grão (milho, ervilha, grão de bico)",2
2,"segunda-feira, 01 setembro de 2025",Saladas,Chuchu colorido\nFruta,3
3,"segunda-feira, 01 setembro de 2025",Saladas,Cenoura cozida,4
4,"segunda-feira, 01 setembro de 2025",Saladas,Repolho cozido\nFruta,5
...,...,...,...,...
85,"segunda-feira, 15 setembro de 2025",Prato Principal,Frango assado ao molho de mostarda,2
86,"segunda-feira, 15 setembro de 2025",Prato Principal,Strogonoff de frango,3
87,"segunda-feira, 15 setembro de 2025",Prato Principal,Bife bovino acebolado,4
88,"segunda-feira, 15 setembro de 2025",Prato Principal,Hambúrguer suíno ao molho tomatelli,5
